In [1]:
import json
from IPython.display import display, Audio

def remove_comments(json_str):
    lines = json_str.split("\n")
    lines_without_comments = [line.split("//")[0] for line in lines]
    return "\n".join(lines_without_comments)

# Ler o arquivo JSON como uma string
with open('config.json', 'r') as file:
    json_content = file.read()

# Remover os comentários
json_content_without_comments = remove_comments(json_content)

# Carregar o JSON
data = json.loads(json_content_without_comments)

In [2]:
data

{'experiment': {'path_name': './experiments/example',
  'description': 'Example of one abstract experiment',
  'seed': 42},
 'data': {'data_dir': '../../data/2024',
  'use_holdout': True,
  'holdout_config': {'save_to': '../../holdout/',
   'k': 61,
   'percent': 0.2,
   'rating_leq_than': 3},
  'sr': 32000,
  'n_fft': 1095,
  'win_len': 412,
  'hop_len': 100,
  'min_freq': 40,
  'max_freq': 15000},
 'preprocessing': {'save_wav_chunks': True,
  'chunks_size': 5,
  'save_to': '../../preprocessed_data/chunks'},
 'model': {'bottleneck': 'efficientnet_b0'},
 'training': {'batch_size': 32,
  'n_workers': 4,
  'optimizer': 'adam',
  'lr': 0.0003,
  'weight_decay': 1e-05,
  'epochs': 5,
  'device': 'cuda',
  'mix_precision': False,
  'aug': {'use_gaussian_noise': False,
   'use_pink_noise': False,
   'use_background_noise': False,
   'background_wavs': './background_noise/',
   'use_horizontal_flip': True,
   'use_xymasking': True,
   'use_mixup': True}}}

# Generating a 5sec pre processed wavs from all previous competition that have same labels

In [4]:
from scipy.io.wavfile import write
import numpy as np
import librosa
import pandas as pd

# data_dir_2024 = '../../data/2024'
data_base = '../../data/'
dfs = {}
for year in ['2021','2022','2023','2024']:
    df_tmp = pd.read_csv(f'{data_base}{year}/train_metadata.csv')
    df_tmp['year'] = int(year)
    dfs[year] = df_tmp


In [22]:
dupes = {
    ("asbfly/XC347563.ogg", "asbfly/XC341611.ogg"),
    ("grewar3/XC507658.ogg", "grewar3/XC184040.ogg"),
    
    ('asbfly/XC724266.ogg', 'asbfly/XC724148.ogg'),
    ('barswa/XC575749.ogg', 'barswa/XC575747.ogg'),
    ('bcnher/XC669544.ogg', 'bcnher/XC669542.ogg'),
    ('bkskit1/XC350251.ogg', 'bkskit1/XC350249.ogg'),
    ('blhori1/XC417215.ogg', 'blhori1/XC417133.ogg'),
    ('blhori1/XC743616.ogg', 'blhori1/XC537503.ogg'),
    ('blrwar1/XC662286.ogg', 'blrwar1/XC662285.ogg'),
    ('brakit1/XC743675.ogg', 'brakit1/XC537471.ogg'),
    ('brcful1/XC197746.ogg', 'brcful1/XC157971.ogg'),
    ('brnshr/XC510751.ogg', 'brnshr/XC510750.ogg'),
    ('btbeat1/XC665307.ogg', 'btbeat1/XC513403.ogg'),
    ('btbeat1/XC743618.ogg', 'btbeat1/XC683300.ogg'),
    ('btbeat1/XC743619.ogg', 'btbeat1/XC683300.ogg'),
    ('btbeat1/XC743619.ogg', 'btbeat1/XC743618.ogg'),
    ('categr/XC787914.ogg', 'categr/XC438523.ogg'),
    ('cohcuc1/XC253418.ogg', 'cohcuc1/XC241127.ogg'),
    ('cohcuc1/XC423422.ogg', 'cohcuc1/XC423419.ogg'),
    ('comgre/XC202776.ogg', 'comgre/XC192404.ogg'),
    ('comgre/XC602468.ogg', 'comgre/XC175341.ogg'),
    ('comgre/XC64628.ogg', 'comgre/XC58586.ogg'),
    ('comior1/XC305930.ogg', 'comior1/XC303819.ogg'),
    ('comkin1/XC207123.ogg', 'comior1/XC207062.ogg'),
    ('comkin1/XC691421.ogg', 'comkin1/XC690633.ogg'),
    ('commyn/XC577887.ogg', 'commyn/XC577886.ogg'),
    ('commyn/XC652903.ogg', 'commyn/XC652901.ogg'),
    ('compea/XC665320.ogg', 'compea/XC644022.ogg'),
    ('comsan/XC385909.ogg', 'comsan/XC385908.ogg'),
    ('comsan/XC643721.ogg', 'comsan/XC642698.ogg'),
    ('comsan/XC667807.ogg', 'comsan/XC667806.ogg'),
    ('comtai1/XC126749.ogg', 'comtai1/XC122978.ogg'),
    ('comtai1/XC305210.ogg', 'comtai1/XC304811.ogg'),
    ('comtai1/XC542375.ogg', 'comtai1/XC540351.ogg'),
    ('comtai1/XC542379.ogg', 'comtai1/XC540352.ogg'),
    ('crfbar1/XC615780.ogg', 'crfbar1/XC615778.ogg'),
    ('dafbab1/XC188307.ogg', 'dafbab1/XC187059.ogg'),
    ('dafbab1/XC188308.ogg', 'dafbab1/XC187068.ogg'),
    ('dafbab1/XC188309.ogg', 'dafbab1/XC187069.ogg'),
    ('dafbab1/XC197745.ogg', 'dafbab1/XC157972.ogg'),
    ('eaywag1/XC527600.ogg', 'eaywag1/XC527598.ogg'),
    ('eucdov/XC355153.ogg', 'eucdov/XC355152.ogg'),
    ('eucdov/XC360303.ogg', 'eucdov/XC347428.ogg'),
    ('eucdov/XC365606.ogg', 'eucdov/XC124694.ogg'),
    ('eucdov/XC371039.ogg', 'eucdov/XC368596.ogg'),
    ('eucdov/XC747422.ogg', 'eucdov/XC747408.ogg'),
    ('eucdov/XC789608.ogg', 'eucdov/XC788267.ogg'),
    ('goflea1/XC163901.ogg', 'bladro1/XC163901.ogg'),
    ('goflea1/XC208794.ogg', 'bladro1/XC208794.ogg'),
    ('goflea1/XC208795.ogg', 'bladro1/XC208795.ogg'),
    ('goflea1/XC209203.ogg', 'bladro1/XC209203.ogg'),
    ('goflea1/XC209549.ogg', 'bladro1/XC209549.ogg'),
    ('goflea1/XC209564.ogg', 'bladro1/XC209564.ogg'),
    ('graher1/XC357552.ogg', 'graher1/XC357551.ogg'),
    ('graher1/XC590235.ogg', 'graher1/XC590144.ogg'),
    ('grbeat1/XC304004.ogg', 'grbeat1/XC303999.ogg'),
    ('grecou1/XC365426.ogg', 'grecou1/XC365425.ogg'),
    ('greegr/XC247286.ogg', 'categr/XC197438.ogg'),
    ('grewar3/XC743681.ogg', 'grewar3/XC537475.ogg'),
    ('grnwar1/XC197744.ogg', 'grnwar1/XC157973.ogg'),
    ('grtdro1/XC651708.ogg', 'grtdro1/XC613192.ogg'),
    ('grywag/XC459760.ogg', 'grywag/XC457124.ogg'),
    ('grywag/XC575903.ogg', 'grywag/XC575901.ogg'),
    ('grywag/XC650696.ogg', 'grywag/XC592019.ogg'),
    ('grywag/XC690448.ogg', 'grywag/XC655063.ogg'),
    ('grywag/XC745653.ogg', 'grywag/XC745650.ogg'),
    ('grywag/XC812496.ogg', 'grywag/XC812495.ogg'),
    ('heswoo1/XC357155.ogg', 'heswoo1/XC357149.ogg'),
    ('heswoo1/XC744698.ogg', 'heswoo1/XC665715.ogg'),
    ('hoopoe/XC631301.ogg', 'hoopoe/XC365530.ogg'),
    ('hoopoe/XC631304.ogg', 'hoopoe/XC252584.ogg'),
    ('houcro1/XC744704.ogg', 'houcro1/XC683047.ogg'),
    ('houspa/XC326675.ogg', 'houspa/XC326674.ogg'),
    ('inbrob1/XC744708.ogg', 'inbrob1/XC744706.ogg'),
    ('insowl1/XC305214.ogg', 'insowl1/XC301142.ogg'),
    ('junbab2/XC282587.ogg', 'junbab2/XC282586.ogg'),
    ('labcro1/XC267645.ogg', 'labcro1/XC265731.ogg'),
    ('labcro1/XC345836.ogg', 'labcro1/XC312582.ogg'),
    ('labcro1/XC37773.ogg', 'labcro1/XC19736.ogg'),
    ('labcro1/XC447036.ogg', 'houcro1/XC447036.ogg'),
    ('labcro1/XC823514.ogg', 'gybpri1/XC823527.ogg'),
    ('laudov1/XC185511.ogg', 'grewar3/XC185505.ogg'),
    ('laudov1/XC405375.ogg', 'laudov1/XC405374.ogg'),
    ('laudov1/XC514027.ogg', 'eucdov/XC514027.ogg'),
    ('lblwar1/XC197743.ogg', 'lblwar1/XC157974.ogg'),
    ('lewduc1/XC261506.ogg', 'lewduc1/XC254813.ogg'),
    ('litegr/XC403621.ogg', 'bcnher/XC403621.ogg'),
    ('litegr/XC535540.ogg', 'litegr/XC448898.ogg'),
    ('litegr/XC535552.ogg', 'litegr/XC447850.ogg'),
    ('litgre1/XC630775.ogg', 'litgre1/XC630560.ogg'),
    ('litgre1/XC776082.ogg', 'litgre1/XC663244.ogg'),
    ('litspi1/XC674522.ogg', 'comtai1/XC674522.ogg'),
    ('litspi1/XC722435.ogg', 'litspi1/XC721636.ogg'),
    ('litspi1/XC722436.ogg', 'litspi1/XC721637.ogg'),
    ('litswi1/XC443070.ogg', 'litswi1/XC440301.ogg'),
    ('lobsun2/XC197742.ogg', 'lobsun2/XC157975.ogg'),
    ('maghor2/XC197740.ogg', 'maghor2/XC157978.ogg'),
    ('maghor2/XC786588.ogg', 'maghor2/XC786587.ogg'),
    ('malpar1/XC197770.ogg', 'malpar1/XC157976.ogg'),
    ('marsan/XC383290.ogg', 'marsan/XC383288.ogg'),
    ('marsan/XC733175.ogg', 'marsan/XC716673.ogg'),
    ('mawthr1/XC455222.ogg', 'mawthr1/XC455211.ogg'),
    ('orihob2/XC557991.ogg', 'orihob2/XC557293.ogg'),
    ('piebus1/XC165050.ogg', 'piebus1/XC122395.ogg'),
    ('piebus1/XC814459.ogg', 'piebus1/XC792272.ogg'),
    ('placuc3/XC490344.ogg', 'placuc3/XC486683.ogg'),
    ('placuc3/XC572952.ogg', 'placuc3/XC572950.ogg'),
    ('plaflo1/XC615781.ogg', 'plaflo1/XC614946.ogg'),
    ('purher1/XC467373.ogg', 'graher1/XC467373.ogg'),
    ('purher1/XC827209.ogg', 'purher1/XC827207.ogg'),
    ('pursun3/XC268375.ogg', 'comtai1/XC241382.ogg'),
    ('pursun4/XC514853.ogg', 'pursun4/XC514852.ogg'),
    ('putbab1/XC574864.ogg', 'brcful1/XC574864.ogg'),
    ('rewbul/XC306398.ogg', 'bkcbul1/XC306398.ogg'),
    ('rewbul/XC713308.ogg', 'asbfly/XC713467.ogg'),
    ('rewlap1/XC733007.ogg', 'rewlap1/XC732874.ogg'),
    ('rorpar/XC199488.ogg', 'rorpar/XC199339.ogg'),
    ('rorpar/XC402325.ogg', 'comior1/XC402326.ogg'),
    ('rorpar/XC516404.ogg', 'rorpar/XC516402.ogg'),
    ('sbeowl1/XC522123.ogg', 'brfowl1/XC522123.ogg'),
    ('sohmyn1/XC744700.ogg', 'sohmyn1/XC743682.ogg'),
    ('spepic1/XC804432.ogg', 'spepic1/XC804431.ogg'),
    ('spodov/XC163930.ogg', 'bladro1/XC163901.ogg'),
    ('spodov/XC163930.ogg', 'goflea1/XC163901.ogg'),
    ('spoowl1/XC591485.ogg', 'spoowl1/XC591177.ogg'),
    ('stbkin1/XC266782.ogg', 'stbkin1/XC266682.ogg'),
    ('stbkin1/XC360661.ogg', 'stbkin1/XC199815.ogg'),
    ('stbkin1/XC406140.ogg', 'stbkin1/XC406138.ogg'),
    ('vefnut1/XC197738.ogg', 'vefnut1/XC157979.ogg'),
    ('vefnut1/XC293526.ogg', 'vefnut1/XC289785.ogg'),
    ('wemhar1/XC581045.ogg', 'comsan/XC581045.ogg'),
    ('wemhar1/XC590355.ogg', 'wemhar1/XC590354.ogg'),
    ('whbbul2/XC335671.ogg', 'whbbul2/XC335670.ogg'),
    ('whbsho3/XC856465.ogg', 'whbsho3/XC856463.ogg'),
    ('whbsho3/XC856468.ogg', 'whbsho3/XC856463.ogg'),
    ('whbsho3/XC856468.ogg', 'whbsho3/XC856465.ogg'),
    ('whbwat1/XC840073.ogg', 'whbwat1/XC840071.ogg'),
    ('whbwoo2/XC239509.ogg', 'rufwoo2/XC239509.ogg'),
    ('whcbar1/XC659329.ogg', 'insowl1/XC659329.ogg'),
    ('whiter2/XC265271.ogg', 'whiter2/XC265267.ogg'),
    ('whtkin2/XC197737.ogg', 'whtkin2/XC157981.ogg'),
    ('whtkin2/XC430267.ogg', 'whtkin2/XC430256.ogg'),
    ('whtkin2/XC503389.ogg', 'comior1/XC503389.ogg'),
    ('whtkin2/XC540094.ogg', 'whtkin2/XC540087.ogg'),
    ('woosan/XC184466.ogg', 'marsan/XC184466.ogg'),
    ('woosan/XC545316.ogg', 'woosan/XC476064.ogg'),
    ('woosan/XC587076.ogg', 'woosan/XC578599.ogg'),
    ('woosan/XC742927.ogg', 'woosan/XC740798.ogg'),
    ('woosan/XC825766.ogg', 'grnsan/XC825765.ogg'),
    ('zitcis1/XC303866.ogg', 'zitcis1/XC302781.ogg'),
}

In [23]:
# Removing duplicates

all_dups = []
for a, b in dupes:
    all_dups.append(a)
    all_dups.append(b)
dfs['2024'] = dfs['2024'][~dfs['2024'].filename.isin(all_dups)]

cols_2024 = list(dfs['2024'].columns)
targets = list(dfs['2024'].primary_label.unique())
len(targets), targets[:3]

(182, ['asbfly', 'ashdro1', 'ashpri1'])

In [24]:
inter_cols = []

for year in dfs.keys():
    if year != '2024':
        filt_df = dfs[year][dfs[year].primary_label.isin(targets)]
        print(f'Dataset from {year} has shape = {filt_df.shape} and have {filt_df.primary_label.nunique()} unique labels!')
        for c in dfs[year].columns:
            if c in cols_2024:
                inter_cols.append(c)
        dfs[year] = dfs[year][dfs[year].primary_label.isin(targets)].reset_index(drop=True)
inter_cols = list(set(inter_cols))
print(f'We have {len(inter_cols)} same columns!')

Dataset from 2021 has shape = (2074, 15) and have 7 unique labels!
Dataset from 2022 has shape = (2545, 14) and have 11 unique labels!
Dataset from 2023 has shape = (4145, 13) and have 14 unique labels!
We have 13 same columns!


In [25]:
df_full = pd.DataFrame()
for year in dfs.keys():
    df_full = pd.concat([df_full, dfs[year][inter_cols]])
df_full = df_full.reset_index(drop=True)
df_full.head()

,license,rating,secondary_labels,primary_label,scientific_name,latitude,common_name,author,longitude,filename,year,type,url
0,Creative Commons Attribution-NonCommercial-Sha...,4.0,[],barswa,Hirundo rustica,29.2670,Barn Swallow,Todd Mark,-108.1098,XC11464.ogg,2021,['call'],https://www.xeno-canto.org/11464
1,Creative Commons Attribution-NonCommercial-Sha...,3.5,[],barswa,Hirundo rustica,54.5789,Barn Swallow,Louis A. Hansen,11.9284,XC125026.ogg,2021,['song'],https://www.xeno-canto.org/125026
2,Creative Commons Attribution-NonCommercial-Sha...,3.5,[],barswa,Hirundo rustica,-28.8865,Barn Swallow,Sander Bot,24.8353,XC126103.ogg,2021,['call'],https://www.xeno-canto.org/126103
3,Creative Commons Attribution-NonCommercial-Sha...,4.5,"['marwre', 'sonspa']",barswa,Hirundo rustica,35.0307,Barn Swallow,Thomas G. Graves,-120.6205,XC129647.ogg,2021,['song'],https://www.xeno-canto.org/129647
4,Creative Commons Attribution-NonCommercial-Sha...,3.0,[],barswa,Hirundo rustica,45.3675,Barn Swallow,Patrick Turgeon,-73.8566,XC132406.ogg,2021,['song'],https://www.xeno-canto.org/132406


In [26]:
df_full.shape, dfs['2024'].shape, df_full.primary_label.nunique()

((32932, 13), (24168, 13), 182)

In [31]:
import os
wpaths = []
for i, fname in enumerate(df_full.filename):
    y = str(df_full.year[i])
    if(y == '2021'):
        wpath = data_base + y + '/train_short_audio/' + df_full['primary_label'][i] +'/' + fname
    else:
        wpath = data_base + y + '/train_audio/' + fname
       
    if(os.path.isfile(wpath)):
        wpaths.append(wpath)
    else:
        print(f'There is no {wpath}')
        break

In [32]:
df_full['wav_path'] = wpaths

In [36]:
## Get durations
from tqdm import tqdm
durs = []
srs = []
for w in tqdm(df_full.wav_path):
    y, sr = librosa.load(w, sr = None)
    dur = len(y)/sr
    durs.append(dur)
    srs.append(sr)

100%|█████████████████████████████████████████████████████████████████████████████| 32932/32932 [23:52<00:00, 22.99it/s]


In [37]:
df_full['sr'] = srs
df_full['duration'] = durs

In [38]:
df_full.sr.unique()

array([32000])

In [40]:
df_full.duration.min(), df_full.duration.max(), df_full.duration.mean()

(0.365, 5964.225, 42.166746536423545)

In [47]:
df_full[df_full.duration>5].shape

(31602, 16)

In [53]:
df_full[df_full.duration>5].primary_label.value_counts()

barswa     1498
houspa     1498
comsan     1493
bcnher     1247
eaywag1     998
           ... 
paisto1       5
niwpig1       5
asiope1       4
darter2       3
integr        2
Name: primary_label, Length: 182, dtype: int64

In [54]:
import librosa
import math
import numpy as np
import scipy

def preprocess_fn(filepath, side = 'center', sr = 32000):

    # LOAD .OGG
    audio_data, _ = librosa.load(filepath, sr=sr)

    if(side == 'center'):
        # CROP CENTER 5 SECONDS,  OR COPY N_COPY TIMES IF <5s (UNTIL 5s is achieved)
        n_copy = math.ceil(5 * sr / len(audio_data))
        if n_copy > 1: audio_data = np.concatenate([audio_data]*n_copy)
        start_idx = int(len(audio_data) / 2 - 2.5 * sr)
        end_idx = int(start_idx + 5.0 * sr)
        input_audio = audio_data[start_idx:end_idx]
    elif(side == 'left'):
        # CROP FIRST 5 SECONDS,  OR COPY N_COPY TIMES IF <5s (UNTIL 5s is achieved)
        n_copy = math.ceil(5 * sr / len(audio_data))
        if n_copy > 1: audio_data = np.concatenate([audio_data]*n_copy)
        start_idx = 0
        end_idx = int(5.0 * sr)
        input_audio = audio_data[start_idx:end_idx]
    elif(side == 'right'):
        # CROP LAST 5 SECONDS,  OR COPY N_COPY TIMES IF <5s (UNTIL 5s is achieved)
        n_copy = math.ceil(5 * sr / len(audio_data))
        if n_copy > 1: audio_data = np.concatenate([audio_data]*n_copy)
        start_idx = int(-5.0 * sr)
        input_audio = audio_data[start_idx:]

    return input_audio

In [60]:
from IPython.display import display, Audio

In [61]:
ex = df_full[df_full['duration'] < 5]['wav_path'].values[0]
chunk = preprocess_fn(ex, side = 'center', sr=32000)
Audio(chunk, rate = 32000)

In [62]:
ex = df_full[df_full['duration'] < 5]['wav_path'].values[-1]
chunk = preprocess_fn(ex, side = 'center', sr=32000)
Audio(chunk, rate = 32000)

# Generating audios

In [64]:
from scipy.io.wavfile import write
import tqdm

In [63]:
df_full['mapped'] = df_full.primary_label.map(df_full.primary_label.value_counts())

In [65]:
df_full.columns

Index(['license', 'rating', 'secondary_labels', 'primary_label',
       'scientific_name', 'latitude', 'common_name', 'author', 'longitude',
       'filename', 'year', 'type', 'url', 'wav_path', 'sr', 'duration',
       'mapped'],
      dtype='object')

In [68]:
!ls ../../preprocessed_data


5sec_unlabeleds_center	baseline_fastvit     rgbimgs_v0
5sec_wavs		baseline_holdout_v1  rgbimgs_v1
5sec_wavs_augmented	imgs_v0		     simple_5sec_spectrogram


In [67]:
# !mkdir ../../preprocessed_data/5sec_wavs_augmented

In [70]:
''.join(('_'.join(df_full.wav_path.values[0].split('/')).split('.')[:-1]))

'__data_2021_train_short_audio_barswa_XC11464'

In [71]:
labels = []
secondary = []
ratings = []
wav_paths = []

for i in tqdm.tqdm(range(df_full.shape[0])):
    sliced_audio = preprocess_fn(df_full['wav_path'].values[i], side = 'center', sr=32000)
    
    slice_name = ''.join(('_'.join(df_full.filename.values[i].split('/')).split('.')[:-1]))
    
    write(f"../../preprocessed_data/5sec_wavs_augmented/{slice_name}.wav", 32000, sliced_audio.astype(np.float32))
    
    wav_paths.append(f"../../preprocessed_data/5sec_wavs_augmented/{slice_name}.wav")
    labels.append(df_full['primary_label'].values[i])
    secondary.append(df_full['secondary_labels'].values[i])
    ratings.append(df_full['rating'].values[i])
    
    if(df_full.mapped.values[i] < 50):
        sliced_audio = preprocess_fn(df_full['wav_path'].values[i], side = 'left', sr=32000)
        slice_name = ''.join(('_'.join(df_full.filename.values[i].split('/')).split('.')[:-1]))
        write(f"../../preprocessed_data/5sec_wavs_augmented/{slice_name}_left.wav", 32000, sliced_audio.astype(np.float32))
        wav_paths.append(f"../../preprocessed_data/5sec_wavs_augmented/{slice_name}.wav")
        labels.append(df_full['primary_label'].values[i])
        secondary.append(df_full['secondary_labels'].values[i])
        ratings.append(df_full['rating'].values[i])
        
        
        sliced_audio = preprocess_fn(df_full['wav_path'].values[i], side = 'right', sr=32000)
        slice_name = ''.join(('_'.join(df_full.filename.values[i].split('/')).split('.')[:-1]))
        write(f"../../preprocessed_data/5sec_wavs_augmented/{slice_name}_right.wav", 32000, sliced_audio.astype(np.float32))
        wav_paths.append(f"../../preprocessed_data/5sec_wavs_augmented/{slice_name}.wav")
        labels.append(df_full['primary_label'].values[i])
        secondary.append(df_full['secondary_labels'].values[i])
        ratings.append(df_full['rating'].values[i])

100%|█████████████████████████████████████████████████████████████████████████████| 32932/32932 [26:31<00:00, 20.69it/s]


In [72]:
new_df = pd.DataFrame({'wav_path': wav_paths, 'primary_label': labels, 'secondary_labels': secondary, 'rating': ratings})
new_df.head()

,wav_path,primary_label,secondary_labels,rating
0,../../preprocessed_data/5sec_wavs_augmented/XC...,barswa,[],4.0
1,../../preprocessed_data/5sec_wavs_augmented/XC...,barswa,[],3.5
2,../../preprocessed_data/5sec_wavs_augmented/XC...,barswa,[],3.5
3,../../preprocessed_data/5sec_wavs_augmented/XC...,barswa,"['marwre', 'sonspa']",4.5
4,../../preprocessed_data/5sec_wavs_augmented/XC...,barswa,[],3.0


In [73]:
import os

for w in new_df.wav_path:
    if(not os.path.isfile(w)):
        print(w)
new_df.to_csv('augmented_only_pl_no_dups_5sec_leq50.csv', index = False)

In [74]:
new_df.shape

(36870, 4)

## OLD

In [4]:
df_filt = df[df['secondary_labels'] == '[]']
df_filt.shape, df.shape

((22567, 13), (24459, 13))

In [5]:
df_filt.primary_label.nunique()

182

In [6]:
df_filt.primary_label.value_counts()

comsan     488
graher1    486
commoo3    485
eurcoo     484
bcnher     484
          ... 
wynlau1      6
malwoo1      6
integr       5
asiope1      5
niwpig1      4
Name: primary_label, Length: 182, dtype: int64

In [7]:
import librosa
import math
import numpy as np
import scipy

def preprocess_fn(filepath, side = 'center', sr = 32000):

    # LOAD .OGG
    audio_data, _ = librosa.load(filepath, sr=sr)

    if(side == 'center'):
        # CROP CENTER 5 SECONDS,  OR COPY N_COPY TIMES IF <5s (UNTIL 5s is achieved)
        n_copy = math.ceil(5 * sr / len(audio_data))
        if n_copy > 1: audio_data = np.concatenate([audio_data]*n_copy)
        start_idx = int(len(audio_data) / 2 - 2.5 * sr)
        end_idx = int(start_idx + 5.0 * sr)
        input_audio = audio_data[start_idx:end_idx]
    elif(side == 'left'):
        # CROP FIRST 5 SECONDS,  OR COPY N_COPY TIMES IF <5s (UNTIL 5s is achieved)
        n_copy = math.ceil(5 * sr / len(audio_data))
        if n_copy > 1: audio_data = np.concatenate([audio_data]*n_copy)
        start_idx = 0
        end_idx = int(5.0 * sr)
        input_audio = audio_data[start_idx:end_idx]
    elif(side == 'right'):
        # CROP LAST 5 SECONDS,  OR COPY N_COPY TIMES IF <5s (UNTIL 5s is achieved)
        n_copy = math.ceil(5 * sr / len(audio_data))
        if n_copy > 1: audio_data = np.concatenate([audio_data]*n_copy)
        start_idx = int(-5.0 * sr)
        input_audio = audio_data[start_idx:]

    return input_audio

In [8]:
processed_spec_left = preprocess_fn(df_filt['filepath'].values[0], side = 'center', sr=32000)

In [9]:
Audio(processed_spec_left, rate = 32000)

In [10]:
Audio(processed_spec_left, rate = 32000)

In [11]:
Audio(processed_spec_left, rate = 32000)

In [12]:
df_filt.filename.nunique(), df_filt.shape

(22567, (22567, 13))

In [14]:
# Removing duplicates

all_dups = []
for a, b in dupes:
    all_dups.append(a)
    all_dups.append(b)

In [15]:
df_filt2 = df_filt[~df_filt.filename.isin(all_dups)]

In [16]:
df_filt2.shape, df_filt.shape, df_filt2.primary_label.nunique()

((22292, 13), (22567, 13), 182)

In [17]:
df_filt2.primary_label.value_counts()

commoo3    485
eurcoo     484
grnsan     482
graher1    481
comsan     481
          ... 
wynlau1      6
malwoo1      6
integr       5
asiope1      5
niwpig1      4
Name: primary_label, Length: 182, dtype: int64

In [18]:
df_filt2['mapped'] = df_filt2.primary_label.map(df_filt2.primary_label.value_counts())

/tmp/ipykernel_1044030/1953644383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt2['mapped'] = df_filt2.primary_label.map(df_filt2.primary_label.value_counts())


In [19]:
df_filt2.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,filepath,mapped
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg,../../data/2024/train_audio/asbfly/XC134896.ogg,91
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg,../../data/2024/train_audio/asbfly/XC164848.ogg,91
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg,../../data/2024/train_audio/asbfly/XC175797.ogg,91
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg,../../data/2024/train_audio/asbfly/XC207738.ogg,91
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg,../../data/2024/train_audio/asbfly/XC209218.ogg,91


In [20]:
''.join(('_'.join(df_filt2.filename.values[0].split('/')).split('.')[:-1]))

'asbfly_XC134896'

# Generating 5 sec audios

In [21]:
!ls ../../preprocessed_data/

5sec_wavs	  baseline_holdout_v1  rgbimgs_v0  simple_5sec_spectrogram
baseline_fastvit  imgs_v0	       rgbimgs_v1


In [22]:
from scipy.io.wavfile import write
import tqdm

In [59]:
labels = []
wav_paths = []

for i in tqdm.tqdm(range(df_filt2.shape[0])):
    sliced_audio = preprocess_fn(df_filt2['filepath'].values[i], side = 'center', sr=32000)
    
    slice_name = ''.join(('_'.join(df_filt2.filename.values[i].split('/')).split('.')[:-1]))
    
    write(f"../../preprocessed_data/5sec_wavs/{slice_name}.wav", 32000, sliced_audio.astype(np.float32))
    
    wav_paths.append(f"../../preprocessed_data/5sec_wavs/{slice_name}.wav")
    labels.append(df_filt2['primary_label'].values[i])
    
    if(df_filt2.mapped.values[i] < 20):
        sliced_audio = preprocess_fn(df_filt2['filepath'].values[i], side = 'left', sr=32000)
        slice_name = ''.join(('_'.join(df_filt2.filename.values[i].split('/')).split('.')[:-1]))
        write(f"../../preprocessed_data/5sec_wavs/{slice_name}_left.wav", 32000, sliced_audio.astype(np.float32))
        wav_paths.append(f"../../preprocessed_data/5sec_wavs/{slice_name}.wav")
        labels.append(df_filt2['primary_label'].values[i])
        
        
        sliced_audio = preprocess_fn(df_filt2['filepath'].values[i], side = 'right', sr=32000)
        slice_name = ''.join(('_'.join(df_filt2.filename.values[i].split('/')).split('.')[:-1]))
        write(f"../../preprocessed_data/5sec_wavs/{slice_name}_right.wav", 32000, sliced_audio.astype(np.float32))
        wav_paths.append(f"../../preprocessed_data/5sec_wavs/{slice_name}.wav")
        labels.append(df_filt2['primary_label'].values[i])

100%|█████████████████████████████████████████████████████████████████████████████| 22292/22292 [15:01<00:00, 24.73it/s]


In [60]:
new_df = pd.DataFrame({'wav_path': wav_paths, 'label': labels})

In [61]:
new_df.shape

(23176, 2)

In [62]:
new_df.head(5)

,wav_path,label
0,../../preprocessed_data/5sec_wavs/asbfly_XC134...,asbfly
1,../../preprocessed_data/5sec_wavs/asbfly_XC164...,asbfly
2,../../preprocessed_data/5sec_wavs/asbfly_XC175...,asbfly
3,../../preprocessed_data/5sec_wavs/asbfly_XC207...,asbfly
4,../../preprocessed_data/5sec_wavs/asbfly_XC209...,asbfly


In [63]:
import os

for w in new_df.wav_path:
    if(not os.path.isfile(w)):
        print(w)

In [64]:
new_df.to_csv('only_pl_no_dups_5sec_leq20.csv', index = False)

# Generating for the unlabelled soundscapes

In [24]:
import os
us = os.listdir('../../data/2024/unlabeled_soundscapes/')

us = ['../../data/2024/unlabeled_soundscapes/' + u for u in us]

us[:2]

['../../data/2024/unlabeled_soundscapes/1311639762.ogg',
 '../../data/2024/unlabeled_soundscapes/1786701443.ogg']

In [27]:
''.join(((us[0].split('/')[-1]).split('.')[:-1]))

'1311639762'

In [28]:
labels = []
wav_paths = []

for i in tqdm.tqdm(range(len(us))):
    sliced_audio = preprocess_fn(us[i], side = 'center', sr=32000)
    
    slice_name = ''.join(((us[i].split('/')[-1]).split('.')[:-1]))
    
    write(f"../../preprocessed_data/5sec_unlabeleds_center/{slice_name}.wav", 32000, sliced_audio.astype(np.float32))
    
    wav_paths.append(f"../../preprocessed_data/5sec_unlabeleds_center/{slice_name}.wav")
#     labels.append(df_filt2['primary_label'].values[i])

 17%|█████████████▋                                                                 | 1467/8444 [05:11<24:42,  4.71it/s]

In [29]:
df = pd.DataFrame('wav_path': wav_paths)

df.to_csv('unlabeled_5sec_center.csv', index = False)